# Setup

In [ ]:
!pip install jupysql
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.9/295.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

# BeautifulSoup

BeautifulSoup é uma biblioteca Python que é usada para web scraping, ou seja, para extrair dados de páginas HTML e XML. É uma ferramenta incrivelmente útil e eficiente que permite ao programador focar em manipular os dados já coletados em vez de ter de lidar com os detalhes da extração de dados.



In [ ]:
# string HTML
html_string = """
<html>
<head>
<title>Sirius ETL</title>
</head>
<body>
<h1>Você está na página Sirius ETL</h1>
<p>ETL significa Extract, Transform, Load, que são as etapas fundamentais para a integração e preparação de dados em um ambiente de análise.</p>
</body>
</html>
"""

In [ ]:
soup = BeautifulSoup(html_string, 'html.parser')

# Criando uma instância BeautifulSoup
soup = BeautifulSoup(html_string, 'html.parser')

# Imprimindo o título da página
print(soup.title.string)

# Imprimindo o primeiro h1 na página
print(soup.h1.string)

# Imprimindo o primeiro parágrafo na página
print(soup.p.string)

Sirius ETL
Você está na página Sirius ETL
ETL significa Extract, Transform, Load, que são as etapas fundamentais para a integração e preparação de dados em um ambiente de análise.


In [ ]:
soup.find_all('p')

[<p>ETL significa Extract, Transform, Load, que são as etapas fundamentais para a integração e preparação de dados em um ambiente de análise.</p>]

# Extração

Vamos utilizar o módulo requests para obter o conteúdo HTML da página e o Beautiful Soup para fazer o parsing desses dados. Neste exemplo, vamos obter uma tabela com dados dos países da América do Sul da página https://pt.wikipedia.org/wiki/Am%C3%A9rica_do_Sul como exemplo.

In [ ]:
# Fazer requisição GET para a página da Wikipedia e obter o conteúdo HTML
url = "https://pt.wikipedia.org/wiki/Am%C3%A9rica_do_Sul"
response = requests.get(url)
html = response.content

# Criar o objeto Beautiful Soup
soup = BeautifulSoup(html, "html.parser")

# Encontrar a tabela desejada na página (pode variar dependendo da página e dados que você deseja extrair)
table = soup.find_all("table", class_="wikitable")[1]

# Extrair os dados da tabela para uma lista
data = []
for row in table.find_all("tr"):
    cols = row.find_all("td")
    if cols:
        values = [col.text.strip() for col in cols]
        data.append(values)

In [ ]:
data

[['Argentina', '2\xa0791\xa0810\xa0km²', '39\xa0745\xa0613', 'Buenos Aires'],
 ['Aruba (\xa0Países Baixos) *', '193\xa0km²', '102\xa0695', 'Oranjestad'],
 ['Bolívia', '1\xa0098\xa0581\xa0km²', '9\xa0627\xa0269', 'La Paz e Sucre'],
 ['Bonaire (\xa0Países Baixos) **', '294\xa0km²', '15\xa0800', 'Kralendijk'],
 ['Brasil', '8\xa0515\xa0767\xa0km²', '200\xa0104\xa0749', 'Brasília'],
 ['Chile', '756\xa0950\xa0km²', '16\xa0598\xa0074', 'Santiago'],
 ['Colômbia', '1\xa0141\xa0748\xa0km²', '44\xa0379\xa0598', 'Bogotá'],
 ['Curaçao (\xa0Países Baixos) *', '444\xa0km²', '142\xa0180', 'Willemstad'],
 ['Equador', '256\xa0370\xa0km²', '13\xa0810\xa0000', 'Quito'],
 ['Guiana', '214\xa0970\xa0km²', '751\xa0000', 'Georgetown'],
 ['Guiana Francesa (\xa0França) **',
  '86\xa0504\xa0km²',
  '209\xa0000',
  'Caiena'],
 ['Ilhas Malvinas (\xa0Reino Unido)*',
  '12\xa0200\xa0km²',
  '3\xa0060',
  'Port Stanley'],
 ['Ilhas Geórgia do Sul e Sandwich do Sul (\xa0Reino Unido) *',
  '4\xa0057\xa0km²',
  '100',
  '

# Transformação

Aqui, podemos utilizar as funcionalidades do pandas para manipular e transformar os dados extraídos da tabela.

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

,0,1,2,3
0,Argentina,2 791 810 km²,39 745 613,Buenos Aires
1,Aruba ( Países Baixos) *,193 km²,102 695,Oranjestad
2,Bolívia,1 098 581 km²,9 627 269,La Paz e Sucre
3,Bonaire ( Países Baixos) **,294 km²,15 800,Kralendijk
4,Brasil,8 515 767 km²,200 104 749,Brasília
5,Chile,756 950 km²,16 598 074,Santiago
6,Colômbia,1 141 748 km²,44 379 598,Bogotá
7,Curaçao ( Países Baixos) *,444 km²,142 180,Willemstad
8,Equador,256 370 km²,13 810 000,Quito
9,Guiana,214 970 km²,751 000,Georgetown


In [ ]:
df.columns = ["País", "Área", "População", "Capital"]

In [ ]:
df

,País,Área,População,Capital
0,Argentina,2 791 810 km²,39 745 613,Buenos Aires
1,Aruba ( Países Baixos) *,193 km²,102 695,Oranjestad
2,Bolívia,1 098 581 km²,9 627 269,La Paz e Sucre
3,Bonaire ( Países Baixos) **,294 km²,15 800,Kralendijk
4,Brasil,8 515 767 km²,200 104 749,Brasília
5,Chile,756 950 km²,16 598 074,Santiago
6,Colômbia,1 141 748 km²,44 379 598,Bogotá
7,Curaçao ( Países Baixos) *,444 km²,142 180,Willemstad
8,Equador,256 370 km²,13 810 000,Quito
9,Guiana,214 970 km²,751 000,Georgetown


In [ ]:
df.set_index('País', inplace=True)

In [ ]:
df

In [ ]:
df.loc['Trindade e Tobago', 'População'] = df.loc['Trindade e Tobago', 'Capital']


In [ ]:
df.loc['Trindade e Tobago', 'Capital'] = df.loc['Trindade e Tobago', 0]

In [ ]:
df.drop(0, axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.drop('5 131 km²', axis=0, inplace=True)

In [ ]:
df

In [ ]:
df["População"] = df["População"].str.replace("\xa0", "")
df["População"] = df["População"].str.replace(" ", "")
df["Área"] = df["Área"].str.replace("km²", "")
df["Área"] = df["Área"].str.replace("\xa0", "")
df["Área"] = df["Área"].str.replace(" ", "")
df["Área"] = df["Área"].str.replace("km2", "")

In [ ]:
df

In [ ]:
df['Área'] = df['Área'].astype(int)
df['População'] = df['Área'].astype(int)

In [ ]:
df.loc[df['Área'].idxmax()]

# Carregamento

Finalmente, podemos utilizar o módulo sqlite3 para criar um banco de dados SQLite3 e salvar o DataFrame como uma tabela dentro dele.


In [ ]:
%load_ext sql

In [ ]:
%%sql sqlite://

In [ ]:
%sql --persist df

In [ ]:
%%sql

select * from df